## Import libraries and dependencies


In [10]:
import fetch_data as fd

In [12]:
# Request crypto tickers from user, and record as list
ticker_list = fd.input_cryptos()

# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = fd.get_ticker_data(ticker_list)

# portfolio_df.head() to view data    

Please enter the tickers of your cryptocurrencies one by one
Type done when finished.


Ticker:  eth
Ticker:  btc
Ticker:  luna
Ticker:  xrp
Ticker:  done


Crpytocurrencies selected:
['ETH', 'BTC', 'LUNA', 'XRP']


Please select the interval: 
(Valid intervals: 1d, 5d, 1wk, 1mo, 3mo)
 1d
